In [1]:
import praw
import pandas as pd
from praw.models import MoreComments
import requests
from datetime import datetime, timedelta

In [2]:
client_id = "UoICzNoOSaQRdRdY38IAnA"
client_secret = "mwR_HDfoFM1muvkktNB-c6PzHHfNkg"
user_agent = "Stock News"

acc = praw.Reddit(client_id=client_id,
                  client_secret=client_secret, 
                  user_agent=user_agent)

ticker = "AAPL"
subreddit = acc.subreddit(ticker)

In [3]:
# Scraping all top posts from 1/1/19 to 31/12/22
start_date = datetime(2019, 1, 1)
end_date = datetime(2022, 12, 31)

posts = subreddit.top(limit=None)

filtered_posts = [post for post in posts 
                  if datetime.utcfromtimestamp(post.created_utc) >= start_date
                  and datetime.utcfromtimestamp(post.created_utc) <= end_date]

posts_dict = {
    "Title": [], 
    "Post Text": [],
    "ID": [], 
    "Score": [],
    "Date": []
}

for post in filtered_posts:
    # Title of each post
    posts_dict["Title"].append(post.title)

    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)

    # Unique ID of each post
    posts_dict["ID"].append(post.id)

    # The score/upvotes of a post
    posts_dict["Score"].append(post.score)
    
    # Date of each post
    unix_timestamp = post.created_utc
    date = datetime.fromtimestamp(unix_timestamp)
    posts_dict["Date"].append(date)

posts = pd.DataFrame(posts_dict).sort_values(by='Date', ascending=False)
posts

,Title,Post Text,ID,Score,Date
441,How are we expecting the Quarterly results to ...,Christmas? Supply problems? Inflation? I know ...,zyhhfv,2,2022-12-30 05:46:14
543,$AAPL could fall below $100 - analyst,,zy4bzk,0,2022-12-29 20:33:01
515,Apple’s Plans for Car and AR Headset Reflect M...,,zj14du,0,2022-12-12 01:40:54
442,AAPL Stock: Sellers Drive The Price Lower But ...,,zeypyv,2,2022-12-07 19:21:29
516,"For anyone holding AAPL, be sure to come THROU...","We are all here for AAPL longterm, please be s...",zdqf7p,0,2022-12-06 09:04:56
...,...,...,...,...,...
331,"Hi All, What a good day for Apple today! I thi...",,ktdt21,6,2021-01-09 06:45:35
491,Is AAPL a good stock to buy now?,With AAPL at $241 is now a good time to jump i...,fum21r,2,2020-04-04 10:51:09
440,Amazing Stock Tip: Tim Cook’s Apple (AAPL) 100...,,eoo3tk,3,2020-01-15 01:09:56
513,"Apple Stock React: iPhone 11, Watch 5 and APPL...",,d43lcy,1,2019-09-14 18:38:50


In [4]:
posts["Date"].dt.year.value_counts()

2021    463
2022    109
2020      2
2019      2
Name: Date, dtype: int64

In [56]:
# posts.to_csv('reddit posts.csv', index = False)

In [58]:
# Get top level comments 
post_ids = posts['ID']
post_comments = []

for post_id in post_ids:
    submission = acc.submission(id=post_id)

    for comment in submission.comments:
        if type(comment) == MoreComments:
            continue
        comment_dict = {'Comment': comment.body, 'Post ID': post_id}    
        post_comments.append(comment_dict)

comments_df = pd.DataFrame(post_comments)

# Drop comments that are removed or deleted
comments_df = comments_df.drop(comments_df[comments_df['Comment'].isin(['[removed]', '[deleted]'])].index)

comments_df

,Comment,Post ID
0,The market is not rational when it comes to th...,zyhhfv
1,Just got back from purchasing an Apple Watch s...,zyhhfv
2,I believe it will be a bloodbath \n\nLong term...,zyhhfv
3,When are they due ?,zyhhfv
4,I'm an analyst and I say it can go above $500 ...,zy4bzk
...,...,...
2610,yea .... you should have bought some,fum21r
2611,"Yes, it was a good time to buy lol",fum21r
2612,It was a very good time. Did you buy?,fum21r
2613,BUY IT. *PUKE.* THEN SELL IT. I don’t know.,fum21r


In [59]:
comments_df['Post ID'].value_counts()

tlg9af    31
uvcpsf    22
og8q8m    21
tph46c    20
pl8n2b    20
          ..
mz1lw4     1
myyp1x     1
rpve3d     1
qecau5     1
cmqtw6     1
Name: Post ID, Length: 517, dtype: int64

In [60]:
# comments_df.to_csv('reddit comments.csv', index = False)